In [2]:
import spacy
import pandas as pd
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Cargar dataset
df = pd.read_csv("/content/drive/Shareddrives/IBD-NLP-RiskPrediction_data/Reddit/processed/reddit_ibd_preprocessed.csv")
df.head()

,id,subreddit,author,title,selftext,created_utc,comments,cuerpo,title_clean,title_lemma,selftext_clean,selftext_lemma,comments_clean,comments_lemma
0,dflwn,CrohnsDisease,zakool21,Don't be afraid of diagnostic procedures....,"I'm not likely to frontpage this subreddit, bu...",2010-09-18 11:59:11,"['I have to agree, the worse part of a Colonos...",TÍTULO:\nDon't be afraid of diagnostic procedu...,do not be afraid of diagnostic procedures,afraid diagnostic procedure,i am not likely to frontpage this subreddit bu...,likely frontpage subreddit glad exist want rea...,i have to agree the worse part of a colonoscop...,agree bad colonoscopy prep glad ok case thank ...
1,dfyy1,CrohnsDisease,sphinctersayzwha,Mayo Clinic article on Crohn's Disease.,Mayo Clinic article on Crohn's Disease.,2010-09-19 15:11:21,[],TÍTULO:\nMayo Clinic article on Crohn's Diseas...,mayo clinic article on crohn s disease,mayo clinic article crohn s disease,mayo clinic article on crohn s disease,mayo clinic article crohn s disease,NaN,NaN
2,dfz1m,CrohnsDisease,WeDeserveDessert,Has anyone else here taken Remicade? What did...,Has anyone else here taken Remicade? What did...,2010-09-19 15:19:54,"[""Remicade (and a bypass) changed my life. Wh...",TÍTULO:\nHas anyone else here taken Remicade? ...,has anyone else here taken remicade what did y...,take remicade think help,has anyone else here taken remicade what did y...,take remicade think help,remicade and a bypass changed my life while it...,remicade bypass change life eventually decreas...
3,dh6zd,CrohnsDisease,RosenTurd,"I was Diagnosed With Crohns Disease at age 9 ,...",Like the title Says AMA,2010-09-22 04:50:57,"[""My son was diagnosed at ten. It's hell for h...",TÍTULO:\nI was Diagnosed With Crohns Disease a...,i was diagnosed with crohns disease at age 9 n...,diagnose crohns disease age 9 28yo ama,like the title says ama,like title say ama,my son was diagnosed at ten it is hell for him...,son diagnose hell colonoscopy hospital visit d...
4,dhgot,CrohnsDisease,unknownpleasures,Has anyone here had to have bowel surgery more...,I had a small bowel resection and appendectomy...,2010-09-22 18:31:36,"[""I have a story:\n\nI had a small bowel resec...",TÍTULO:\nHas anyone here had to have bowel sur...,has anyone here had to have bowel surgery more...,bowel surgery,i had a small bowel resection and appendectomy...,small bowel resection appendectomy 6 year ago ...,i have a story n ni had a small bowel resectio...,story n ni small bowel resection appendectomy ...


In [8]:
# Cargar modelo spaCy
nlp = spacy.load("en_core_web_sm")

In [9]:
# Procesar textos
texts = df["selftext_lemma"].fillna("").tolist()

ner_results = []

for text in tqdm(texts, desc="Applying NER"):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    ner_results.append(entities)

df["entities"] = ner_results

# Guardar dataset con entidades
df.to_csv("/content/drive/Shareddrives/IBD-NLP-RiskPrediction_data/Reddit/processed/reddit_ibd_with_entities.csv", index=False)

Applying NER: 100%|██████████| 183744/183744 [33:18<00:00, 91.96it/s]


In [10]:
from collections import Counter
import itertools

all_entities = list(itertools.chain.from_iterable(df["entities"]))
labels = [label for _, label in all_entities]

Counter(labels).most_common(10)

[('DATE', 241861),
 ('CARDINAL', 103328),
 ('ORG', 52006),
 ('TIME', 30544),
 ('PERSON', 23558),
 ('ORDINAL', 12835),
 ('GPE', 9411),
 ('QUANTITY', 8701),
 ('NORP', 6550),
 ('PRODUCT', 3017)]

In [22]:
# Ejemplo con entidades resaltadas
import random
from spacy import displacy

i = random.randint(0, len(df)-1)
doc = nlp(df.iloc[i]["selftext_lemma"])
displacy.render(doc, style="ent", jupyter=True)

In [23]:
%pip install scispacy
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_md-0.5.4.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 116.1 MB/s eta 0:00:00
  Created wheel for en_core_sci_md: filename=en_core_sci_md-0.5.4-py3-none-any.whl size=119157918 sha256=8192229d0456faa8f5978b455764516ac335ca1bd9ac67d15f7ca4dda826d8f8
  Stored in directory: /root/.cache/pip/wheels/1a/02/d9/4d4bda80f6b73c02ed057d4b3b99abaff286924a80b53c9d68
Successfully built en_core_sci_md
  Attempting uninstall: blis
    Found existing installation: blis 1.2.1
    Uninstalling blis-1.2.1:
      Successfully uninstalled blis-1.2.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.4
    Uninstalling thinc-8.3.4:
      Successfully uninstalled thinc-8.3.4
  Attempting uninstall: spacy
    Fou

In [4]:
# Cargar modelo spaCy
nlp = spacy.load("en_core_sci_md")

/usr/local/lib/python3.12/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [5]:
# Procesar textos
texts = df["selftext_lemma"].fillna("").tolist()

ner_results = []

for text in tqdm(texts, desc="Applying NER"):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    ner_results.append(entities)

df["entities"] = ner_results

# Guardar dataset con entidades
df.to_csv("/content/drive/Shareddrives/IBD-NLP-RiskPrediction_data/Reddit/processed/reddit_ibd_with_entities_scispacy.csv", index=False)

Applying NER: 100%|██████████| 183744/183744 [37:16<00:00, 82.17it/s]


In [6]:
from collections import Counter
import itertools

all_entities = list(itertools.chain.from_iterable(df["entities"]))
labels = [label for _, label in all_entities]

Counter(labels).most_common(10)

[('ENTITY', 2331327)]

In [12]:
# Ejemplo con entidades resaltadas
import random
from spacy import displacy

i = random.randint(0, len(df)-1)
doc = nlp(df.iloc[i]["selftext_lemma"])
displacy.render(doc, style="ent", jupyter=True)